 ###  `declare` ~ `variant`  Directive

 
A  `declare variant`  directive specifies that the following function is an alternate function,  
a  _function variant_ , to be used in place of the specified  _base function_   
when the trait within the  `match`  clause has a valid context.

 A  `declare` ~ `variant`  directive specifies an alternate function,   _function variant_ , to be used in place of the  _base function_   
when the trait within the  `match`  clause has a valid context. when the trait within the  `match`  clause matches the OpenMP context at a given call site. The base function follows the directive in the C and C++ languages. In Fortran, either a subroutine or function may be used as the  _base function_ , and the  `declare` ~ `variant`  directive must be in the specification  part of a subroutine or function (unless a  _base-proc-name_  modifier is used, as in the case of a procedure declaration statement). See the OpenMP 5.0 Specification for details on the modifier.

 When multiple  `declare` ~ `variant`  directives are used  a function variant becomes a candidate for replacing the base function if the 
base function call context matches the traits of all selectors in the  `match`  clause. context at the base function call matches the traits of all selectors in the  `match`  clause. If there are multiple candidates, a score is assigned with rules for each of the selector traits. The scoring algorithm can be found in the OpenMP 5.0 Specification.

 In the first example the  _vxv()_  function is called within a  `parallel`  region, a  `target`  region, and in a sequential part of the program.  Two function variants,  _p_vxv()_  and  _t_vxv()_ , are defined for the first two regions by using  _parallel_  and  _target_  selectors (within the  _construct_  trait set) in a  `match`  clause.  The  _p_vxv()_  function variant includes a  `for`  construct ( `do`  construct for Fortran) for the  `parallel`  region,  while  _t_vxv()_  includes a  `distribute` ~ `simd`  construct for the  `target`  region. The  _t_vxv()_  function is explicitly compiled for the device using a  `declare` ~ `target`  directive.

 Since the two  `declare` ~ `variant`  directives have no selectors that match traits for the context of the base function call in the sequential part of the program, the base  _vxv()_  function is used there,  as expected. (The vectors in the  _p_vxv_  and  _t_vxv_  functions have been multiplied by 3 and 2, respectively, for checking the validity of the replacement. Normally the purpose of a function variant is to produce the same results by a different method.)

 
Note: a  `target teams`  construct is used to direct execution onto a device, with a 
 `distribute simd`  construct in the function variant. As of the OpenMP 5.0 implementation 
no intervening  is allowed between a  `target`  and  `teams`  construct. So 
using a  `target`  construct to direct execution onto a device, and including  
 `teams distribute simd`  in the variant function would produce non conforming .

 
cexample[5.0]{declare_variant}{1}

In [ ]:

! name: declare_variant.1f90
! type: F-free
! version: omp_5.0

module subs
  use omp_lib
contains
   subroutine vxv(v1, v2, v3)             !! base function
      integer,intent(in)  :: v1(:),v2(:)
      integer,intent(out) :: v3(:)
      integer             :: i,n
      !$omp  declare variant( p_vxv ) match( construct={parallel} )
      !$omp  declare variant( t_vxv ) match( construct={target}   )

      n=size(v1)
      do i = 1,n; v3(i) = v1(i) * v2(i); enddo

   end subroutine

   subroutine p_vxv(v1, v2, v3)            !! function variant
      integer,intent(in)  :: v1(:),v2(:)
      integer,intent(out) :: v3(:)
      integer             :: i,n
      n=size(v1)

      !$omp do
      do i = 1,n; v3(i) = v1(i) * v2(i) * 3; enddo

   end subroutine

   subroutine t_vxv(v1, v2, v3)            !! function variant
      integer,intent(in)  :: v1(:),v2(:)
      integer,intent(out) :: v3(:)
      integer             :: i,n
      !$omp declare target
      n=size(v1)

      !$omp distribute simd
      do i = 1,n; v3(i) = v1(i) * v2(i) * 2; enddo

   end subroutine

end module subs


program main
   use omp_lib
   use subs
   integer,parameter :: N = 100
   integer           :: v1(N), v2(N), v3(N)

   do i= 1,N; v1(i)= i; v2(i)= -i; v3(i)= 0;  enddo  !! init

   !$omp parallel
      call vxv(v1,v2,v3)
   !$omp end parallel
   print *, v3(1),v3(N)    !! from p_vxv -- output: -3  -30000

   !$omp target teams map(to: v1,v2) map(from: v3)
      call vxv(v1,v2,v3)
   !$omp end target teams
   print *, v3(1),v3(N)    !! from t_vxv -- output: -2  -20000

   call vxv(v1,v2,v3)
   print *, v3(1),v3(N)    !! from   vxv -- output: -1  -10000

end program



 In this example, traits from the  _device_  set are used to select a function variant. In the  `declare` ~ `variant`  directive, an  _isa_  selector specifies that if the implementation of the " _core-avx512_ ''  instruction set is detected at compile time the  _avx512_saxpy()_  variant function is used for the call to  _base_saxpy()_ .  

 A compilation of  _avx512_saxpy()_  is aware of the AVX-512 instruction set that supports 512-bit vector extensions (for Xeon or Xeon Phi architectures).  Within  _avx512_saxpy()_ , the  `parallel` ~ `for` ~ `simd`  construct performs parallel execution, and takes advantage of 64-byte data alignment.  When the  _avx512_saxpy()_  function variant is not selected, the base  _base_saxpy()_  function variant containing only a basic  `parallel` ~ `for`  construct is used for the call to  _base_saxpy()_ .

 
Note: 
An allocator is used to set the alignment to 64 bytes when an OpenMP compilation is performed.   
Details about allocator variable declarations and functions 
can be found in the allocator example of the Memory Management Chapter.

 
cexample[5.0]{declare_variant}{2}

In [ ]:

! name: declare_variant.2f90
! type: F-free
! version: omp_5.0

module subs
  use omp_lib
contains

   subroutine base_saxpy(s,x,y)              !! base function
      real,intent(inout) :: s,x(:),y(:)
     !$omp  declare variant( avx512_saxpy ) &
     !$omp&           match( device={isa("core-avx512")} )

      y = s*x + y

   end subroutine

   subroutine avx512_saxpy(s,x,y)               !! function variant
      real,intent(inout) :: s,x(:),y(:)
      integer            :: i,n
      n=size(x)
                             !!assume 64-byte alignment for AVX-512
      !$omp parallel do simd simdlen(16) aligned(x,y: 64)
      do i = 1,n
         y(i) = s*x(i) + y(i)
      end do

   end subroutine

end module subs


program main
   use omp_lib
   use subs

   integer, parameter :: N=1000, align=64
   real, allocatable  :: x(:),y(:)
   real               :: s = 2.0e0
   integer            :: i

   allocate(x(N),y(N))   !! Assumes allocation is 64-byte aligned
                         !! (using compiler options, or another
                         !! allocation method).

                         !! loc is non-standard, but found everywhere
                         !! remove these lines if not available
   if(modulo(loc(x),align) /= 0 .and. modulo(loc(y),align) /=0 ) then
      print*,"ERROR: x|y not 64-byte aligned"; stop
   endif

   do i=1,N  !! initialize
     x(i)=i
     y(i)=i
   end do

   call base_saxpy(s,x,y)

   write(*,'("y(1),y(N):",2f6.0)') y(1),y(N) !!output: y... 3. 3000.

   deallocate(x,y)

end program

